# Calculate page growth, per engagement

This notebook calculates each pages' growth, in terms of changes in monthly engagement between March 2016 and March 2017.

In [1]:
import pandas as pd
import datetime

## Load the aggregated data

In [2]:
monthly_engagement = pd.read_csv("../output/page-engagement-by-month.csv", dtype={"page_id": str})

## Merge the two comparison months

In [3]:
comparison = pd.merge(
    monthly_engagement[
        monthly_engagement["status_published"] == "2016-03-01"
    ][[ "page_id", "total_engagement" ]],
    monthly_engagement[
        monthly_engagement["status_published"] == "2017-03-01"
    ][[ "page_id", "total_engagement" ]],
    on="page_id",
    how="outer",
    suffixes=[ ".march2016", ".march2017" ]
)
comparison.head()

,page_id,total_engagement.march2016,total_engagement.march2017
0,100434040001314,52080.0,55029.0
1,1019871961378419,519875.0,1950091.0
2,103971336305218,96008.0,126475.0
3,104037195986,3905.0,3673.0
4,10498053716,419447.0,601169.0


## Add political category and page name information

In [4]:
page_info = pd.read_csv(
    "../output/fb-page-info-summary.csv",
    dtype={"page_id": str}
)
page_info.head()

,page_id,political_category,page_name,fan_count
0,100434040001314,left,Wonkette,96322
1,1014803551921469,right,SourcesNews,59306
2,1019871961378419,right,Rare America,2982929
3,1035617169863710,right,I DID NOT Vote For Hillary in 2016,36269
4,1036253643101134,left,Proud Liberal,510991


In [5]:
def categorize_growth(page):
    diff = page["total_engagement.march2017"] - page["total_engagement.march2016"]
    if diff > 0:
        return "grown_{political_category}".format(**page)
    elif diff == 0:
        return "flat_{political_category}".format(**page)
    else:
        return "shrunken_{political_category}".format(**page)

In [6]:
comparison_with_fb_info = pd.merge(
    comparison,
    page_info,
    on="page_id",
    how="left"
).assign(growth_category=lambda x: x.fillna(0).apply(categorize_growth, axis=1))
comparison_with_fb_info.head()

,page_id,total_engagement.march2016,total_engagement.march2017,political_category,page_name,fan_count,growth_category
0,100434040001314,52080.0,55029.0,left,Wonkette,96322,grown_left
1,1019871961378419,519875.0,1950091.0,right,Rare America,2982929,grown_right
2,103971336305218,96008.0,126475.0,right,Hot Air,820318,grown_right
3,104037195986,3905.0,3673.0,left,Black Agenda Report,23332,shrunken_left
4,10498053716,419447.0,601169.0,right,Media Research Center,1728105,grown_right


In [7]:
# Make sure we have partisanship categorizations for each page
assert comparison_with_fb_info["political_category"].isnull().sum() == 0

In [8]:
comparison_with_fb_info.to_csv("../output/page-engagement-growth.csv", index=False)

## Pare down growth to top 20 pages on left and right  for graphics
This part of the notebook takes the top 20 pages on the left and right and pares down the growth spreadsheet to those top 20.

In [9]:
top_20s = comparison_with_fb_info.groupby("political_category")\
    .apply(lambda x: x.nlargest(20, "fan_count"))
top_20s

page_id  total_engagement.march2016  \
political_category                                                     
left               212   341163402640457                  22751278.0   
                   218   354522044588660                   8231964.0   
                   215   346937065399354                  26394599.0   
                   19    114517875225866                  12244729.0   
                   95    160389977329803                   4421326.0   
                   357  1340186812659728                         NaN   
                   328   979613892126968                   1470734.0   
                   175       24674986856                    258183.0   
                   428   610045389164725                         NaN   
                   184   273864989376427                   2292659.0   
                   150      210277954204                   2136335.0   
                   138   200137333331078                   1288136.0   
                   122      177486166274                   5160854.0   
                   378  1672814509645693                         NaN   
                   299        7642602143                    784004.0   
                   104  1640832309490921                   3145538.0   
                   391   188464111175168                         NaN   
                   40    129370207168068                   1035011.0   
                   50    139398939438430                    533627.0   
                   64    146422995398181                   3844800.0   
right              87        15704546335                  27600038.0   
                   17    114364638621903                   2856203.0   
                   128   184859785026060                   6216784.0   
                   124   179035672287016                  16394130.0   
                   31       123624513983                   3232290.0   
                   126      182919686769                   3200779.0   
                   136  1969250316633715                   3178355.0   
                   277       58158120046                   1900579.0   
                   262   519305544814653                   2512689.0   
                   42    133961323610549                   8284190.0   
                   229   389658314427637                   7038986.0   
                   362   145380305523723                         NaN   
                   323       95475020353                   9066729.0   
                   141   201956993160690                   1126835.0   
                   290       69813760388                   2306415.0   
                   426   592074077639509                         NaN   
                   15    112723252096438                   8294477.0   
                   1    1019871961378419                    519875.0   
                   112   169204449790211                   1917771.0   
                   282        6333396177                    931395.0   

                        total_engagement.march2017 political_category  \
political_category                                                      
left               212                  18783790.0               left   
                   218                   5459343.0               left   
                   215                  36673027.0               left   
                   19                   19753039.0               left   
                   95                    5793239.0               left   
                   357                  10240180.0               left   
                   328                   5961902.0               left   
                   175                    282455.0               left   
                   428                   2371347.0               left   
                   184                   2075205.0               left   
                   150                   2322745.0               left   
                   138                   1288656.0         

In [10]:
top_20s.loc["left"].to_csv("../output/page-engagement-growth-top-20-pages-left.csv", index=False)
top_20s.loc["right"].to_csv("../output/page-engagement-growth-top-20-pages-right.csv", index=False)

---

---

---